In [75]:
!pip install pyinterval

In [76]:
from interval import interval
import pandas as pd
import math

In [77]:
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_middle_point(interv):
    return (interv[0].sup + interv[0].inf) / 2

def to_string(interv):
    return f"[{interv[0].inf:9.7f}, {interv[0].sup:9.7f}]"

In [78]:
def calculate_volterra2(f, expected, k, kdf, y0, a, b, c, d, n):
    h = (b - a) / n
    x = []
    y = []
    exp = []
    x.append(a)
    y.append(y0)
    exp.append(expected(a))
    for i in range(1, n+1):
        x.append(a + i*h)
        sum = 0
        for j in range(i):
            sum += k(x[i], x[j], y[j])
        y.append(f(x[i]) + h*sum + (h*(x[i]-a)/2)*kdf(x[i], interval[a, x[i]], interval[c, d]))
        exp.append(expected(x[i]))
    return x, y, exp

In [79]:
a = 0
b = math.sqrt(3)-1
c = 0
d = 2
n = 5
y0 = interval[c,d]
func = lambda x: x**2
k = lambda x,s,y : (-s)*y
kdf = lambda x,s,y : -y
expected = lambda x: 2-2*math.exp(-x**2/2)

# Riemann sum

In [80]:
tbl2 = pd.DataFrame(columns=['xi', 'yi', 'm(yi)', 'expected', 'e'])
x, y, exp = calculate_volterra2(func, expected, k, kdf, y0, a, b, c, d, n)
for i in range(n+1):
    mid_point=calculate_middle_point(y[i])
    tbl2.loc[len(tbl2)] = [f"{x[i]:9.4f}", to_string(y[i]), f"{mid_point:9.7f}", f"{exp[i]:9.7f}", abs(mid_point-exp[i])]
tbl2

,xi,yi,m(yi),expected,e
0,0.0000,"[0.0000000, 2.0000000]",1.0000000,0.0000000,1.000000
1,0.1464,"[-0.0000000, 0.0214359]",0.0107180,0.0213215,0.010604
2,0.2928,"[0.0424124, 0.0857437]",0.0640781,0.0839317,0.019854
3,0.4392,"[0.1244801, 0.1911051]",0.1577926,0.1839107,0.026118
4,0.5856,"[0.2408062, 0.3331516]",0.2869789,0.3151784,0.028199
5,0.7321,"[0.3837280, 0.5054274]",0.4445777,0.4701067,0.025529
